In [602]:
import pandas as pd
import numpy as np

In [603]:
data = pd.read_csv("engish_headlines_3_nnew.csv", encoding="latin-1")

In [604]:
model = open("config.txt", "r").readlines()[1]
path = open("config.txt", "r").readlines()[0]

In [605]:
data

,Unnamed: 0,Date,Headline,begins_at,open_price,close_price
0,0,2/25/2018,Original Pizza Day Purchaser Does It Again Wit...,2/25/2018,9680.2000,9584.45000
1,1,2/25/2018,This 11-year-old just wrote a book on bitcoin ...,2/25/2018,9680.2000,9584.45000
2,2,2/25/2018,"Without Mentioning Blockchain, Putin Says That...",2/25/2018,9680.2000,9584.45000
3,3,2/25/2018,Meet the strippers tattooed with BARCODES so s...,2/25/2018,9680.2000,9584.45000
4,4,2/26/2018,Bitcoin Pizza Day 2: How A Lightning Payment M...,2/26/2018,9592.4950,10318.78500
...,...,...,...,...,...,...
16152,16165,2/24/2023,"Top 3 Price Prediction Bitcoin, Ethereum, Ripp...",2/24/2023,23945.4039,23185.71626
16153,16166,2/24/2023,Bitcoin price stays under $24K as PCE data hel...,2/24/2023,23945.4039,23185.71626
16154,16167,2/24/2023,What are Bitcoin [BTC] whales up to? Decodingâ¦,2/24/2023,23945.4039,23185.71626
16155,16168,2/24/2023,Bitcoin 2024 halving will be its âmost impor...,2/24/2023,23945.4039,23185.71626


In [606]:
headlines = data["Headline"]
close_prices = data[["Date", "open_price"]]
open_prices = data[["Date", "open_price"]]

In [607]:
sentiments_data = pd.read_csv(path[:-1])

In [608]:
if model == "finbert":
    sentiments_data.loc[sentiments_data["Sentiment"] == "Positive", "Sentiment"] = 1
    sentiments_data.loc[sentiments_data["Sentiment"] == "Negative", "Sentiment"] = -1
    sentiments_data.loc[sentiments_data["Sentiment"] == "Neutral", "Sentiment"] = 0
elif model == "gpt":
    sentiments_data.loc[sentiments_data["Labels"] == "Positive", "Labels"] = 1
    sentiments_data.loc[sentiments_data["Labels"] == "Positive.", "Labels"] = 1
    sentiments_data.loc[sentiments_data["Labels"] == "Positive ", "Labels"] = 1
    sentiments_data.loc[sentiments_data["Labels"] == "Negative", "Labels"] = -1
    sentiments_data.loc[sentiments_data["Labels"] == "Negative.", "Labels"] = -1
    sentiments_data.loc[sentiments_data["Labels"] == "Negative ", "Labels"] = -1
    sentiments_data.loc[sentiments_data["Labels"] == "Negative  ", "Labels"] = -1
    sentiments_data.loc[sentiments_data["Labels"] == "Neutral", "Labels"] = 0
    sentiments_data.loc[sentiments_data["Labels"] == "Neutral.", "Labels"] = 0
    sentiments_data.loc[sentiments_data["Labels"] == "Neutral ", "Labels"] = 0
    sentiments_data.loc[sentiments_data["Labels"] == "Uncertain", "Labels"] = 0

    range1 = sentiments_data["Labels"] != 1
    range2 = sentiments_data["Labels"] != 0
    range3 = sentiments_data["Labels"] != -1

    range4 = range1 & range2 & range3

    sentiments_data["Labels"][range4] = 0
else:
    sentiments_data.loc[sentiments_data["Labels"] == "Pos", "Sentiment"] = 1
    sentiments_data.loc[sentiments_data["Labels"] == "Neg", "Sentiment"] = -1
    sentiments_data.loc[sentiments_data["Labels"] == "Ne", "Sentiment"] = 0

In [609]:
sentiments_data

,Labels,Date,Sentiment
0,Pos,2/25/2018,1.0
1,Pos,2/25/2018,1.0
2,Pos,2/25/2018,1.0
3,Ne,2/25/2018,0.0
4,Pos,2/26/2018,1.0
...,...,...,...
16152,Pos,2/24/2023,1.0
16153,Neg,2/24/2023,-1.0
16154,Pos,2/24/2023,1.0
16155,Pos,2/24/2023,1.0


In [610]:
model

'llama'

In [611]:
sentiments_data['Date'] = pd.to_datetime(sentiments_data['Date'])
open_prices['Date'] = pd.to_datetime(open_prices['Date'])

sentiments_data.sort_values('Date', inplace=True)

dates_unique = pd.to_datetime(open_prices['Date'].unique())
x = []
previous = 0
period_days = 1
open_prices_list = []

for index in range(period_days - 1, len(dates_unique), period_days):
    dates_unique_chunk = dates_unique[previous:index+1]
    if not len(dates_unique_chunk):
        continue

    after_start_date = sentiments_data['Date'] >= dates_unique_chunk[0]
    before_end_date = sentiments_data['Date'] <= dates_unique_chunk[-1]
    between_two_dates = after_start_date & before_end_date
    data_chunk = sentiments_data.loc[between_two_dates]

    if model== "gpt":
        sentiments_chunk = data_chunk['Labels']
    else:
        sentiments_chunk = data_chunk['Sentiment']

    open_price = open_prices[open_prices["Date"]==dates_unique_chunk[-1]].iloc[0][1]
    open_prices_list.append(open_price)

    between_range = sentiments_chunk < 0.5
    between_range = between_range & sentiments_chunk > -0.5

    sentiments_chunk_neutral = sentiments_chunk[between_range]
    sentiments_chunk_negative = sentiments_chunk[sentiments_chunk >= 0.5]
    sentiments_chunk_positive = sentiments_chunk[sentiments_chunk <= -0.5]

    n_positive = len(sentiments_chunk_positive)
    n_negative = len(sentiments_chunk_negative)
    n_neutral = len(sentiments_chunk_neutral)

    n = len(sentiments_chunk)

    period_sentiment = sum(sentiments_chunk)/len(sentiments_chunk)
    if period_sentiment > 0.5:
        class_ = 1
    elif period_sentiment < -0.5:
        class_ = -1
    else:
        class_ = 0

    x.append([period_sentiment, n_positive, n_negative, n_neutral])

    previous += period_days

C:\Users\Roman Kypybida\AppData\Local\Temp\ipykernel_17668\1472472109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  open_prices['Date'] = pd.to_datetime(open_prices['Date'])
C:\Users\Roman Kypybida\AppData\Local\Temp\ipykernel_17668\1472472109.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  open_price = open_prices[open_prices["Date"]==dates_unique_chunk[-1]].iloc[0][1]


In [612]:
x

[[0.75, 0, 3, 4],
 [0.4444444444444444, 2, 6, 9],
 [-0.1, 4, 3, 10],
 [0.2, 4, 6, 10],
 [0.0, 5, 5, 10],
 [0.5, 2, 7, 10],
 [0.2857142857142857, 2, 4, 7],
 [0.6666666666666666, 1, 5, 6],
 [-0.4, 7, 3, 10],
 [0.2, 4, 6, 10],
 [-0.2, 6, 4, 10],
 [0.4, 3, 7, 10],
 [0.0, 5, 5, 10],
 [0.0, 1, 1, 2],
 [0.3, 3, 6, 10],
 [-0.1, 5, 4, 10],
 [0.6, 2, 8, 10],
 [-0.5555555555555556, 7, 2, 9],
 [0.2, 4, 6, 10],
 [0.1, 4, 5, 10],
 [0.5555555555555556, 1, 6, 9],
 [0.5, 2, 6, 8],
 [0.4, 2, 6, 10],
 [-0.1, 5, 4, 10],
 [0.4, 3, 7, 10],
 [0.2, 4, 6, 10],
 [0.6, 2, 8, 10],
 [-0.3333333333333333, 2, 1, 3],
 [0.5, 1, 4, 6],
 [0.3, 1, 4, 10],
 [0.2, 4, 6, 10],
 [0.4, 2, 6, 10],
 [0.3, 3, 6, 10],
 [0.14285714285714285, 3, 4, 7],
 [0.3333333333333333, 1, 2, 3],
 [0.2, 2, 3, 5],
 [0.7, 1, 8, 10],
 [0.2, 4, 6, 10],
 [0.6, 2, 8, 10],
 [0.5, 2, 7, 10],
 [0.3, 3, 6, 10],
 [0.5714285714285714, 1, 5, 7],
 [0.42857142857142855, 1, 4, 7],
 [0.7, 1, 8, 10],
 [0.1111111111111111, 4, 5, 9],
 [0.6, 2, 8, 10],
 [0.1, 4, 5, 

In [613]:
y = np.array(open_prices_list)[1:] - np.array(open_prices_list)[:-1]

In [614]:
y

array([ -87.705  ,  726.29   ,  233.685  , ..., -396.40969, -268.66826,
       -242.59777])

In [615]:
y_perc = np.divide(y, np.array(open_prices_list)[:-1])*100

In [616]:
y_perc[:100]

array([ -0.90602467,   7.57143996,   2.26465616,  -2.25070529,
         7.95906724,  -0.98653549,   3.8689634 ,   0.3198816 ,
        -0.79752396,  -5.94480631,  -7.48681683,  -6.21067878,
        -0.6753161 ,  -4.86937905,   8.52963715,  -4.29399872,
         0.16726988, -10.36117858,   0.72058784,   0.08781657,
        -5.00357312,   4.43236605,   4.86552722,   3.6225524 ,
        -0.15576175,  -2.06049949,   2.42035657,  -4.33686823,
        -0.93056336,  -3.60949688,  -4.35423046,   1.8067038 ,
       -10.66013354,  -3.55713023,   1.32433237,  -1.59208411,
         3.44322081,   5.17823076,  -8.38844137,  -0.21609094,
        -2.44850919,   4.31184741,   1.77797648,  -3.7183119 ,
         0.98509127,   1.60842479,  13.98025439,  -0.25113063,
         1.38742136,   4.37544231,  -3.55766098,  -1.99931063,
         3.47872542,   1.2819869 ,   7.1093108 ,   0.70572813,
        -1.37600253,   1.6076125 ,   7.92794809,  -8.18941389,
         4.70519161,  -3.75728434,   4.68974705,   0.56

In [617]:
max(y_perc)

19.711593664864342

In [618]:
np.mean(y_perc)

0.12338201494725282

In [619]:
min(y_perc)

-39.03227734044527

In [620]:
len(x)

1824

In [621]:
y_class = y_perc.copy()
y_class

array([-0.90602467,  7.57143996,  2.26465616, ..., -1.59501235,
       -1.09854801, -1.00296739])

In [622]:
between_range_1 = y_perc >=-1
between_range_2 = y_perc <= 1
between_range = between_range_1 & between_range_2

In [623]:
y_class[between_range] = 0

In [624]:
y_class[between_range]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [625]:
y_class[y_class==0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [626]:
y_class[y_perc < -0.25] = -1

In [627]:
y_class[y_perc > 0.25] = 1

In [628]:
y_perc[y_perc==0.25]

array([], dtype=float64)

In [629]:
dates_unique_list = dates_unique.to_numpy()

In [630]:
dates_unique_list

array(['2018-02-25T00:00:00.000000000', '2018-02-26T00:00:00.000000000',
       '2018-02-27T00:00:00.000000000', ...,
       '2023-02-22T00:00:00.000000000', '2023-02-23T00:00:00.000000000',
       '2023-02-24T00:00:00.000000000'], dtype='datetime64[ns]')

In [631]:
from sklearn.linear_model import RidgeClassifier

In [632]:
reg = RidgeClassifier(alpha=0)

In [633]:
reg.fit(x[1:], y_class)

RidgeClassifier(alpha=0)

In [634]:
reg.score(x[1:], y_class)

0.5167306637410861

In [635]:
predictions_training = reg.predict(x[1:])

In [636]:
predictions_training

array([ 1., -1., -1., ..., -1., -1., -1.])

In [637]:
predictions_training[predictions_training < 0] = -1

In [638]:
predictions_training[predictions_training > 0] = 1

In [639]:
predictions_training[predictions_training == 0] = 0

In [640]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [641]:
confusion_matrix(y_class, predictions_training)

array([[362,   0, 445],
       [ 58,   0, 109],
       [269,   0, 580]], dtype=int64)

In [642]:
accuracy_score(y_class, predictions_training)

0.5167306637410861

In [643]:
f1_score(y_class, predictions_training,average="weighted")

0.48666754156925984

In [644]:
np.savetxt(model + "_reg_" + path.split("/")[0] + ".txt", predictions_training)

In [645]:
from sklearn.ensemble import RandomForestClassifier

In [646]:
clf = RandomForestClassifier(random_state=0)

In [647]:
clf.fit(x[1:], y_class)

RandomForestClassifier(random_state=0)

In [648]:
predictions_training_clf = clf.predict(x[1:])

In [649]:
confusion_matrix(y_class, predictions_training_clf)

array([[524,   0, 283],
       [ 88,   6,  73],
       [375,   1, 473]], dtype=int64)

In [650]:
accuracy_score(y_class, predictions_training_clf)

0.5501919912232583

In [651]:
f1_score(y_class, predictions_training_clf, average="weighted")

0.5274710117546726

In [652]:
np.savetxt(model + "_clf_" + path.split("/")[0] + ".txt", predictions_training_clf)

In [653]:
type(x)

list

In [654]:
from xgboost import XGBClassifier

In [655]:
model_clf = XGBClassifier(objective="multi:softmax")

In [656]:
model_clf

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softmax', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)

In [657]:
model_clf.fit(np.array(x[1:]), y_class)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [658]:
predictions_training_xgb = model_clf.predict(np.array(x[1:]))

In [659]:
predictions_training_xgb

array([-1.,  1., -1., ..., -1.,  1., -1.])

In [660]:
confusion_matrix(y_class, predictions_training_xgb, labels=[-1,0,1])

array([[503,   0, 304],
       [ 84,   6,  77],
       [355,   1, 493]], dtype=int64)

In [661]:
accuracy_score(y_class, predictions_training_xgb)

0.5496434448710916

In [662]:
f1_score(y_class, predictions_training_xgb, average="weighted")

0.5274486985034124

In [663]:
type(predictions_training_xgb)

numpy.ndarray

In [664]:
path.split("/")[0]

'Llama7B'

In [665]:
np.savetxt(model + "_xgb_" + path.split("/")[0] + ".txt", predictions_training_xgb)